In [6]:
import requests
import urllib
from bs4 import BeautifulSoup
import time
import numpy as np
import re
import pandas as pd
import json

#### Import ASIN data
- ASIN stands for Amazon Standard Identification Number
- It is like an unique ID of each item sold in Amazon
- ASIN data for rugs, the product is rug

In [7]:
import pandas as pd
items = pd.read_csv('amazon category A.csv')
items = items.drop_duplicates("(Parent) ASIN")
items

,(Parent) ASIN,(Child) ASIN,Units Ordered,Well Woven SKU
0,B01D5JGOO8,B01D5KC4SC,526,VI41-5
1,B01B3VML60,B01B3VAMVG,410,600165
2,B071X12KKP,B072QSPMLG,358,AR-2031
3,B015AQARDI,B015AMT378,319,9535
4,B01H7X6HEA,B01H7X6OS4,276,6630-2031
6,B01H7X6GJ6,B01H7X7CW6,259,6518-2L
7,B071GGRX7Z,B072MW656V,247,VE-80-5
10,B00SOULB38,B01BJH8H52,221,547805
12,B079YYB1B3,B01M0OP5XF,204,MC-62-5
13,B016FNZ4D8,B0181DGGKA,177,36002


#### Get a list of ASIN

In [8]:
ASIN = items['(Child) ASIN'].tolist()
ASIN.remove("B01M0OP5XF")

#### Scrape review data for products
*** Features ***
- Review data
- Review text
- Review author
- Size
- Color

*** Return ***
- DataFrame: "Review","Star", and "Feature" column
- "Review": customer reviews
- "Star": star of attitute
- "Feature": features of product

#### Get review page URL

In [9]:
review_url = []
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'}
def get_review_url(product_ID):
    base_url = "http://www.amazon.com/dp/"
    
    for num in product_ID:
        url = base_url + num
        response = requests.get(url,headers = headers)
        page = BeautifulSoup(response.content, "html.parser")
        link = page.find_all("a",{"class" : "a-link-emphasis a-text-bold"}, href = True)[0]['href']
        review_url.append(link)

In [10]:
get_review_url(ASIN)

In [11]:
print(len(review_url))
review_url[15] ## test

19


'/Well-Woven-Abstract-Geometric-Neutral/product-reviews/B01I0IYV9O/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'

#### Access to review pages

In [12]:
#if star_div.text != "4.0 out of 5 stars" and '5.0 out of 5 stars':

In [13]:
## build function
def clawer_detail_page(review_data):
    review_body = []
    star_body = []
    review_title_body = []
    review_author_body = []
    review_date_body = []
    size_color_body = []
    
    for data in review_data:
          #star
        star_div = data.find_all('span', {'class' : "a-icon-alt" })[0]
        #print(star_div.text.strip())
        star_body.append(star_div.text.strip())
        
       
            
        #review title
        title_div = data.find_all('a', {'class' : "a-size-base a-link-normal review-title a-color-base a-text-bold"})[0]
        #print(title_div.text.strip()) 
        review_title_body.append(title_div.text.strip())

        #review date
        date_div = data.find_all('span', {'class' : "a-size-base a-color-secondary review-date"})[0]
        #print(date_div.text.strip()) 
        review_date_body.append(date_div.text.strip())

        #review author
        author_div = data.find_all('a', {'class' : "a-size-base a-link-normal author" })[0]
        #print(author_div.text.strip())
        review_author_body.append(author_div.text.strip())

        #size_color

        size_div = data.find_all('a', {"class":"a-size-mini a-link-normal a-color-secondary"})
        if size_div != []:
            size_color_body.append(size_div[0].text.strip())
        else:
            size_color_body.append("None")

        # positive_review
        body_div = data.find_all('span', {'class' : "a-size-base review-text" })[0]
        #print(body_div.text.strip())
        review_body.append(body_div.text.strip())
        
        
    return {"Star":  star_body, "Author":review_author_body,"Title":review_title_body,
           "Date":review_date_body,"Size_Color":size_color_body,"Review":review_body}

#### get review list for each product

In [14]:
def get_review_list(url, pageNum):
    current_url = "https://www.amazon.com"+url+"&pageNumber="+str(pageNum)
    response = requests.get(current_url,headers = headers)
    page = BeautifulSoup(response.content, "html.parser")
    data = page.find_all("div",{"class" : "a-section review"})
    return data

#### get all reviews for all products

In [15]:
review_dict = {}
for ind in range(0,len(review_url)):
    print(review_url[ind])
    num = 1
    reviews = []
    while True:
        print(num)
        data_list = get_review_list(review_url[ind], num)
        if len(data_list) == 0:
            break
        reviews.append(clawer_detail_page(data_list))
        num += 1
    review_dict[ASIN[ind]] = reviews

/Geometric-Abstract-Resistant-Contemporary-Painting/product-reviews/B01D5KC4SC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
/Circles-Geometric-Resistant-Abstract-Contemporary/product-reviews/B01B3VAMVG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
/Well-Woven-Artificial-Resistant-Synthetic/product-reviews/B072QSPMLG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
1
2
3
/Well-Woven-Squares-Geometric-Bathroom/product-reviews/B015AMT378/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
/Well-Woven-Non-Skid-Antibacterial-Traditional/product-reviews/B01H7X6OS4/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
1
2
3
4
5
6
7
8
9
10
11
12
13
14
/Well-Woven-Non-

In [16]:
## review_dict = {ASIN:[{"author":[],"Date":[],...},{},....,{}]}
## len(review_dict['B01B3VAMVG']):47 个 {}

# def get_data(author,title,date,size_color,star,review):
Final_data = {}
for key,value in review_dict.items():
    Author = []
    Title = []
    Date = []  
    Size_Color = []
    Star = []
    Review = []
    for data in value:
        Author.extend(data['Author'])
        Title.extend(data['Title'])
        Date.extend(data['Date'])
        Size_Color.extend(data['Size_Color'])
        Star.extend(data['Star'])
        Review.extend(data['Review'])
    Final_data[key]= [{"Author":Author,"Date":Date,"Star":Star,"Size_Color":Size_Color,
                 "Title":Title,"Review":Review}]

In [17]:
#check
len(Final_data['B013KX2NDK'][0]["Author"])
keys = list(Final_data.keys()) #keys: ASIN
values = list(Final_data.values()) # values: a list of features for each ASIN
len(values)

19

In [18]:
# Df = items[items["(Parent) ASIN"] != "B079YYB1B3"]
# Df

In [19]:
def get_data(key_word):
    data = [Final_data[word][0][key_word] for word in keys]
    return data

In [20]:
# get all review data
Review_data = get_data('Review')
# get all author data
Author_data = get_data('Author')
# get all star data
Star_data = get_data('Star')
# get all size / color data
feature_data = get_data('Size_Color')

In [127]:
len(Review_data[0])

387

In [128]:
reviews = []
for each_review in Review_data:
    for text in each_review:
        reviews.append(text)

In [129]:
stars = []
for each_star in Star_data:
    for text in each_star:
        stars.append(text)

In [131]:
Authors = []
for each_author in Author_data:
    for text in each_author:
        Authors.append(text)

In [132]:
features = []
for each_feature in feature_data:
    for text in each_feature:
        features.append(text)

In [133]:
## final dataframe
final_data = pd.DataFrame({"Review":reviews,"Star":stars,"Feature":features})